<a href="https://colab.research.google.com/github/joepareti54/joepareti54/blob/main/lm_rag_gpt2_test5b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries
!pip install pymupdf sentence-transformers faiss-gpu transformers

import fitz  # PyMuPDF for handling PDFs
import os
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
from google.colab import drive
import warnings
warnings.filterwarnings('ignore')

# Diagnostic functions
def check_drive_mounting():
    """
    Verifies that Google Drive is properly mounted and accessible.
    Returns: bool indicating success
    """
    try:
        drive_path = '/content/drive'
        if not os.path.exists(drive_path):
            print("Google Drive is not mounted!")
            return False
        print("Google Drive is mounted successfully")
        return True
    except Exception as e:
        print(f"Error checking drive mount: {str(e)}")
        return False

def check_directory_content(directory_path):
    """
    Analyzes and reports the contents of the specified directory.
    Args:
        directory_path: str, path to check
    Returns: bool indicating if directory is accessible and contains PDFs
    """
    try:
        if not os.path.exists(directory_path):
            print(f"Directory does not exist: {directory_path}")
            return False

        files = os.listdir(directory_path)
        pdf_files = [f for f in files if f.endswith('.pdf')]

        print(f"\nDirectory contents:")
        print(f"Total files: {len(files)}")
        print(f"PDF files: {len(pdf_files)}")
        if pdf_files:
            print("First few PDF files:")
            for pdf in pdf_files[:5]:
                print(f"- {pdf}")
        return bool(pdf_files)
    except Exception as e:
        print(f"Error checking directory: {str(e)}")
        return False

def extract_text_from_pdf(pdf_path):
    """
    Extracts text content from a PDF file.
    Args:
        pdf_path: str, path to PDF file
    Returns: str, extracted text
    """
    try:
        doc = fitz.open(pdf_path)
        text = ' '.join(page.get_text() for page in doc)
        doc.close()
        return text.strip()
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {str(e)}")
        return ""

def load_and_process_documents(directory_path, limit=15):
    """
    Loads and processes PDF documents from the specified directory.
    Args:
        directory_path: str, path to directory containing PDFs
        limit: int, maximum number of documents to load
    Returns: list of processed document texts
    """
    documents = []

    print("\nChecking drive and directory...")
    if not check_drive_mounting():
        return documents

    if not check_directory_content(directory_path):
        return documents

    print("\nAttempting to load documents...")
    files = [f for f in os.listdir(directory_path) if f.endswith('.pdf')]

    for i, filename in enumerate(files):
        if i >= limit:
            break

        try:
            pdf_path = os.path.join(directory_path, filename)
            print(f"\nProcessing {filename}...")

            if not os.path.exists(pdf_path):
                print(f"File not found: {pdf_path}")
                continue

            text = extract_text_from_pdf(pdf_path)

            if not text:
                print(f"No text extracted from {filename}")
                continue

            if len(text.strip()) < 100:
                print(f"Warning: Very short text ({len(text)} chars) from {filename}")

            documents.append(text)
            print(f"Successfully loaded: {filename} ({len(text)} characters)")

        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")

    print(f"\nDocument loading summary:")
    print(f"Total documents loaded: {len(documents)}")
    if documents:
        print(f"Average document length: {sum(len(d) for d in documents)/len(documents):.0f} characters")

    return documents

def create_embeddings(texts):
    """
    Creates embeddings for the provided texts using Sentence Transformer.
    Args:
        texts: list of str, texts to embed
    Returns: numpy array of embeddings
    """
    return embed_model.encode(texts, show_progress_bar=True)

def preprocess_text(text, max_tokens=250):
    """
    Preprocesses text to fit within token limits.
    Args:
        text: str, input text
        max_tokens: int, maximum number of tokens
    Returns: str, preprocessed text
    """
    tokens = tokenizer.encode(text)
    if len(tokens) > max_tokens:
        tokens = tokens[:max_tokens]
        text = tokenizer.decode(tokens)
    return text

def retrieve_and_generate(query, k=5):
    """
    Retrieves relevant documents and generates a response based on the query.
    Args:
        query: str, the user's question
        k: int, number of documents to retrieve
    Returns: str, generated response
    """
    print(f"\nProcessing query: {query}")

    # Generate query embedding
    query_embedding = create_embeddings([query])[0]
    k = min(k, len(documents))

    # Retrieve similar documents
    distances, indices = index.search(np.array([query_embedding]).astype('float32'), k)

    # Build context from retrieved documents
    retrieved_texts = []
    total_tokens = 0
    max_context_tokens = 500

    print("\nRetrieved documents:")
    for i, idx in enumerate(indices[0]):
        text = documents[idx]
        preview = text[:100] + "..."
        print(f"{i+1}. Score: {distances[0][i]:.4f}\nPreview: {preview}\n")

        # Preprocess and truncate text
        processed_text = preprocess_text(text)
        tokens = tokenizer.encode(processed_text)

        if total_tokens + len(tokens) > max_context_tokens:
            print(f"Reached token limit after {i} documents")
            break

        retrieved_texts.append(processed_text)
        total_tokens += len(tokens)

    context = " ".join(retrieved_texts)

    print(f"Debug - Retrieved texts count: {len(retrieved_texts)}")
    print(f"Debug - Total tokens in context: {total_tokens}")

    # Construct prompt
    combined_text = (
        f"Based on the provided articles, give a concise summary about {query}\n\n"
        f"Articles:\n{context}\n\n"
        "Summary:"
    )

    # Tokenize
    input_ids = tokenizer.encode(
        combined_text,
        truncation=True,
        max_length=1024,
        padding=False,
        return_tensors='pt'
    )

    print(f"Debug - Final input shape: {input_ids.shape}")

    # Move to GPU if available
    input_ids = input_ids.to(device)
    attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=device)

    try:
        outputs = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_new_tokens=150,
            num_beams=3,
            no_repeat_ngram_size=3,
            pad_token_id=tokenizer.eos_token_id,
            early_stopping=True,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.9,
            length_penalty=1.0,
            repetition_penalty=1.2
        )

        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract only the generated summary
        if "Summary:" in generated_text:
            response = generated_text.split("Summary:")[-1].strip()
        else:
            response = generated_text.strip()

        return response

    except Exception as e:
        print(f"Error during generation: {str(e)}")
        return f"Error generating response: {str(e)}"

def test_system(custom_queries=None):
    """
    Tests the system with predefined or custom queries.
    Args:
        custom_queries: list of str, optional custom queries to test
    """
    default_queries = [
        "What are the recent developments in Russia's economy and sanctions?",
        "What's happening with the stock market and gold prices?",
        "What are the latest developments in cryptocurrency?",
        "What's happening with global supply chains and manufacturing?",
        "How are international relations affecting the economy?"
    ]

    test_queries = custom_queries if custom_queries else default_queries

    print("\nTesting system with queries...")
    for query in test_queries:
        print("\n" + "="*80)
        print(f"Query: {query}")
        try:
            response = retrieve_and_generate(query, k=3)
            print(f"\nResponse: {response}")
        except Exception as e:
            print(f"Error processing query: {str(e)}")
        print("="*80)

def main():
    """
    Main execution function that sets up and runs the system.
    """
    global documents, embed_model, tokenizer, model, device, index

    print("Starting document processing and model initialization...")

    # Mount Google Drive
    drive.mount('/content/drive')

    # Define directory path
    directory_path = '/content/drive/My Drive/All_Finance_PDF_files_old/'

    # Load documents
    documents = load_and_process_documents(directory_path)
    if not documents:
        raise Exception("No documents loaded!")

    # Initialize models
    print("\nInitializing models...")
    embed_model = SentenceTransformer('all-MiniLM-L6-v2')
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    tokenizer.pad_token = tokenizer.eos_token

    # Setup GPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    print(f"Using device: {device}")

    # Create embeddings and FAISS index
    print("\nCreating embeddings...")
    embeddings = create_embeddings(documents)

    print("\nSetting up FAISS index...")
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(embeddings).astype('float32'))

    # Run system test
    test_system()

    # Interactive mode
    while True:
        try:
            query = input("\nEnter your query (or 'quit' to exit): ")
            if query.lower() == 'quit':
                break

            response = retrieve_and_generate(query, k=3)
            print(f"\nResponse: {response}")

        except KeyboardInterrupt:
            print("\nExiting interactive mode...")
            break
        except Exception as e:
            print(f"Error: {str(e)}")

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"Fatal error: {str(e)}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.3 MB/s eta 0:00:00
Starting document processing and model initialization...
Mounted at /content/drive

Checking drive and directory...
Google Drive is mounted successfully

Directory contents:
Total files: 2673
PDF files: 2672
First few PDF files:
- China’s Covid-19 Surge Shuts Down Plants in Manufacturing Hubs Shenzhen and Changchun - WSJ.pdf
- Russian Prosecutors Warn Western Companies of Arrests, Asset Seizures - WSJ.pdf
- Können russische Oligarchen Sanktionen mit Kryptowährungen umgehen.pdf
- TikTok Influencers Get Spotlight in Information Battle Over the Russia-Ukraine War - WSJ.pdf
- Chip Makers Stockpiled Key Materials Ahead of Russian Invasion of Ukraine - WSJ.pdf

Attempting to load documents...

Processing China’s Covid-19 Surge Shuts Down Plants in Manufacturing Hubs Shenzhen and Changchun - WSJ.pdf...
Successfully loaded: China’s Covid

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...


Retrieved documents:
1. Score: 1.1460
Preview: This copy is for your personal, non-commercial use only. To order presentation-ready copies for dist...

2. Score: 1.2321
Preview: Manezhnaya Square in Moscow. Analysts expect Russia’s economy to contract as much as 20% this quarte...

3. Score: 1.3907
Preview: DOW JONES, A NEWS CORP COMPANY
About WSJ
POLITICS
Bitcoin Price Surges on Biden’s Crypto Executive O...

Reached token limit after 2 documents
Debug - Retrieved texts count: 2
Debug - Total tokens in context: 500
Debug - Final input shape: torch.Size([1, 528])

Response: The United States and European Union imposed sanctions on Russia over the Ukraine crisis,
and the Russian government responded with a series of measures designed to weaken the
U.S.-led NATO alliance. The sanctions were aimed at preventing Russia from expanding its influence in
the region, and to prevent the United States from using its military might to
escalate the conflict.
Russia's economy has been crippled by t

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Retrieved documents:
1. Score: 1.0721
Preview: DOW JONES, A NEWS CORP COMPANY
About WSJ
BUSINESS
China’s Covid-19 Surge Shuts Down Plants in Manufa...

2. Score: 1.1527
Preview: Manezhnaya Square in Moscow. Analysts expect Russia’s economy to contract as much as 20% this quarte...

3. Score: 1.2566
Preview: This copy is for your personal, non-commercial use only. To order presentation-ready copies for dist...

Reached token limit after 2 documents
Debug - Retrieved texts count: 2
Debug - Total tokens in context: 500
Debug - Final input shape: torch.Size([1, 524])

Response: Russia's economy contracted by more than 20% in the first quarter of this year, according to data from the International Monetary Fund.
Russia is the world's second-largest exporter of crude oil, after the United States, and its

oil production is expected to grow by at least 3.5% this year. It is also the third-largest producer of natural gas

in the world after China and the United Arab Emirates.
In the first hal

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Retrieved documents:
1. Score: 1.2917
Preview: DOW JONES, A NEWS CORP COMPANY
About WSJ
POLITICS
Bitcoin Price Surges on Biden’s Crypto Executive O...

2. Score: 1.3996
Preview: Dow Fell, Oil and Gold Surged—and What
Else Happened in the Stock Market Today
By
Jack Denton
Follow...

3. Score: 1.4250
Preview: The Dow Jones Industrial Average slid nearly 800 points Monday, marking a 10% decline
from January’s...

Reached token limit after 2 documents
Debug - Retrieved texts count: 2
Debug - Total tokens in context: 500
Debug - Final input shape: torch.Size([1, 528])

Response: The Dow Jones industrial average (DJIA) is up 4.3% over the past

week. The Nasdaq composite (DJI) is down 2.9%. The

US dollar is down 1.2%. The Euro is down 0.7%. The Japanese yen

is down 1%. The Russian ruble is up 0.5%. The Chinese yuan is up

0.4% and the Chinese renminbi is up 1.3%.
The Nikkei 225 is up 2.5% and is up 3.1%

on the day. The Shanghai Composite (S&P) has

tied up 1% against the US dollar. The J